In [1]:
import torch, torchvision
from torchvision.transforms import functional as TF
from PIL import Image
import numpy as np
import cv2, os
from pathlib import Path
from tqdm import tqdm
import shutil, itertools

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
# Load COCO-pre-trained Mask R-CNN
model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")
model.to(device).eval()

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [3]:
@torch.inference_mode()
def mask_humans_pil(
    img: Image.Image,
    score_thresh: float = 0.5,
    fill_mode: str = "mean",   # "mean" | "black" | "gray"
):
    """
    Return a copy of `img` where every pixel inside a detected person
    is replaced by the frame average (paper-style) or a solid colour.
    """
    tensor = TF.to_tensor(img).to(device).unsqueeze(0)        # 1×3×H×W
    out = model(tensor)[0]                                    # dict

    keep = (out["labels"] == 1) & (out["scores"] > score_thresh)
    if keep.sum() == 0:
        return img.copy()                                     # no humans

    # Union of all person masks
    masks = (out["masks"][keep, 0].sigmoid() > 0.5).any(0)    # H×W bool
    mask_np = masks.cpu().numpy()

    arr = np.array(img)
    if fill_mode == "mean":
        fill_value = arr.mean(axis=(0, 1), keepdims=True).astype(arr.dtype)
    elif fill_mode == "gray":
        fill_value = np.array([127, 127, 127], dtype=arr.dtype)
    else:  # "black"
        fill_value = 0
    arr[mask_np] = fill_value
    return Image.fromarray(arr)


In [38]:
from itertools import islice

def chunks(iterable, n):
    "Yield successive n-sized lists from iterable"
    it = iter(iterable)
    while True:
        batch = list(islice(it, n))
        if not batch:
            break
        yield batch

def mask_folder_batch(
    in_root: str | Path,
    out_root: str | Path,
    *,
    ext=".jpg",
    score_thresh=0.5,
    batch_size=4,
    recursive=True,
    clear_out=True,
    fill_mode="mean",           # mean / black / gray
):
    """
    Same API as mask_folder, but runs Mask R-CNN on <batch_size> frames
    at once for higher throughput.
    """
    in_root, out_root = Path(in_root), Path(out_root)
    if clear_out and out_root.exists():
        shutil.rmtree(out_root)

    pattern = f"**/*{ext}" if recursive else f"*{ext}"
    files = sorted(in_root.glob(pattern))
    print(f"{len(files)} frame(s) found → processing in batches of {batch_size}")

    for batch_paths in tqdm(list(chunks(files, batch_size))):
        # -------- 1. load & stack tensors ------------------------------------
        imgs_pil   = [Image.open(p).convert("RGB") for p in batch_paths]
        tensors    = [TF.to_tensor(im) for im in imgs_pil]
        tensor_cat = torch.stack(tensors).to(device)          # B×3×H×W

        # -------- 2. forward pass -------------------------------------------
        with torch.inference_mode():
            outs = model(tensor_cat)

        # -------- 3. post-process each frame in the batch -------------------
        for src_path, img, out in zip(batch_paths, imgs_pil, outs):
            keep = (out["labels"] == 1) & (out["scores"] > score_thresh)
            if keep.sum() == 0:
                img_masked = img
            else:
                masks = (out["masks"][keep, 0].sigmoid() > 0.5).any(0).cpu().numpy()
                arr = np.array(img)
                if fill_mode == "mean":
                    fill_val = arr.mean(axis=(0, 1), keepdims=True).astype(arr.dtype)
                elif fill_mode == "gray":
                    fill_val = np.array([127, 127, 127], dtype=arr.dtype)
                else:  # black
                    fill_val = 0
                arr[masks] = fill_val
                img_masked = Image.fromarray(arr)

            dst = out_root / src_path.relative_to(in_root)
            dst.parent.mkdir(parents=True, exist_ok=True)
            img_masked.save(dst)


In [39]:
in_root  = "datasets/UCF-101-JPG/Archery/"            # e.g. ".../train_frames"
out_root = "masking/"          # e.g. ".../train_frames_masked"

mask_folder_batch(
    in_root,
    out_root,
    batch_size = 8,     # tune for your GPU
    score_thresh = 0.55
)

23795 frame(s) found → processing in batches of 8


  0%|          | 0/2975 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [3]:
# ── 0. Setup ────────────────────────────────────────────────────────────────
import torch, torchvision, time, shutil, queue, threading
from torchvision.io import read_file, decode_jpeg, write_jpeg
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# ── 1. Fast JPEG -> Tensor on the GPU (nvJPEG) ──────────────────────────────
class FrameDS(Dataset):
    def __init__(self, root, ext=".jpg"):
        self.paths = sorted(Path(root).rglob(f"*{ext}"))
    def __len__(self): return len(self.paths)
    def __getitem__(self, idx):
        p   = self.paths[idx]
        buf = read_file(str(p))                                   # raw bytes
        img = decode_jpeg(buf, device=device).float() / 255       # C×H×W
        return img, str(p)

# ── 2. Mask-R-CNN (person only) in half precision ──────────────────────────
model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")
model.to(device).eval()

@torch.inference_mode()
def mask_batch(batch_imgs, score=0.5):
    with torch.cuda.amp.autocast(dtype=torch.float16):
        outs = model(batch_imgs)

    rect_masks = []
    for out, im in zip(outs, batch_imgs):
        keep = (out["labels"] == 1) & (out["scores"] > score)
        if keep.sum() == 0:
            rect_masks.append(None)
            continue

        H, W = im.shape[-2:]
        m = torch.zeros((H, W), dtype=torch.bool, device=im.device)
        for box in out["boxes"][keep]:
            x1, y1, x2, y2 = box.round().to(torch.long)
            x1, y1 = x1.clamp(0, W - 1), y1.clamp(0, H - 1)
            x2, y2 = x2.clamp(0, W - 1), y2.clamp(0, H - 1)
            m[y1:y2 + 1, x1:x2 + 1] = True
        rect_masks.append(m)
    return rect_masks

# ── 3. Async disk writer (so saving doesn’t block the GPU) ─────────────────
def writer_thread(q, out_root):
    while True:
        item = q.get()
        if item is None: break
        img, src_path = item
        dst = out_root / Path(src_path).relative_to(in_root)
        dst.parent.mkdir(parents=True, exist_ok=True)
        # uint8 tensor expected by write_jpeg
        write_jpeg((img.clamp(0,1)*255).byte().cpu(), str(dst), quality=95)
        q.task_done()

# ── 4. One-liner “train-loop” that ties it all together ────────────────────
def mask_folder_gpu(in_root, out_root,
                    batch_size=8, workers=4, score=0.5):

    ds  = FrameDS(in_root)
    dl  = DataLoader(ds, batch_size=batch_size, num_workers=workers,
                     pin_memory=False, shuffle=False, drop_last=False)

    if out_root.exists(): shutil.rmtree(out_root)
    q = queue.Queue(maxsize=32)
    th = threading.Thread(target=writer_thread, args=(q, out_root), daemon=True)
    th.start()

    t0 = time.time()
    for imgs, paths in tqdm(dl, total=len(dl)):
        imgs = imgs.to(device, non_blocking=True)
        masks = mask_batch(list(imgs), score)

        # fill on GPU
        for i, m in enumerate(masks):
            if m is not None:
                mean        = imgs[i].mean(dim=(1, 2), keepdim=True)   # (3,1,1)
                mean_pixel  = mean.view(3)        # (3,)
                mean_pixel  = mean_pixel[:, None] # (3,1)  ← add axis so (3,1) → (3,N)
                imgs[i][:, m] = mean_pixel        # OK
        # enqueue for compression + write
        for im, p in zip(imgs, paths): q.put((im, p))

    q.join(); q.put(None); th.join()
    print(f"Done in {(time.time()-t0):.1f}s → {len(ds)/(time.time()-t0):.1f} fps")

In [5]:
# ── 5. Run it ───────────────────────────────────────────────────────────────
in_root  = Path("datasets/UCF-101-JPG/Archery/")            # e.g. ".../train_frames"
out_root = Path("masking/")          # e.g. ".../train_frames_masked"

mask_folder_gpu(in_root, 
                out_root,
                batch_size=8,   # ↑ until you hit ~90 % GPU util or OOM
                workers=0,      # CPU threads that prefetch & decode bytes
                score=0.2)

  0%|          | 0/2975 [00:00<?, ?it/s]

Done in 647.5s → 36.8 fps


## Numpy mask

In [ ]:
import torch, torchvision, numpy as np
from torchvision.io import read_image
from pathlib import Path
from tqdm.auto import tqdm

# ─── config ────────────────────────────────────────────────────────────────
video_root   = Path("datasets/UCF-101-JPG")          # class/video/frame.jpg tree
out_file     = Path("detection_train_merged_rearranged.npy")
score_thr    = 0.2
batch_size   = 8
device       = "cuda" if torch.cuda.is_available() else "cpu"

# ─── detector ──────────────────────────────────────────────────────────────
model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")
model.to(device).eval().half()

@torch.inference_mode()
def detect_batch(imgs):
    with torch.cuda.amp.autocast(dtype=torch.float16):
        return model(imgs)

# ─── build nested dict  {class}{vid11}{frame}{'human_boxes'} ───────────────
all_dets = {}

for class_dir in tqdm(sorted(video_root.iterdir()), desc="class"):
    if not class_dir.is_dir():
        continue
    cls_name = class_dir.name
    all_dets[cls_name] = {}

    for vid_dir in tqdm(sorted(class_dir.iterdir()), leave=False, desc=cls_name):
        if not vid_dir.is_dir():
            continue
        vid11 = vid_dir.name[:11]                   # first 11 chars
        frames = sorted(vid_dir.glob("*.jpg"))
        T = len(frames)
        all_dets[cls_name][vid11] = {}

        # mini-batch for speed
        for i in range(0, T, batch_size):
            batch_paths = frames[i:i+batch_size]
            imgs_cpu    = [read_image(str(p)).float()/255 for p in batch_paths]
            imgs_gpu    = torch.stack(imgs_cpu).to(device).half()
            outs = detect_batch(imgs_gpu)

            for j, out in enumerate(outs, start=i):
                keep = (out["labels"] == 1) & (out["scores"] > score_thr)
                if keep.sum() == 0:
                    boxes = np.zeros((0, 5), dtype=np.float32)
                else:
                    b = out["boxes"][keep].float().cpu().numpy()       # (N,4)
                    s = out["scores"][keep].float().cpu().numpy()[:,None]
                    boxes = np.hstack([b, s]).astype(np.float32)       # (N,5)
                frame_idx = j + 1                                      # 1-based
                all_dets[cls_name][vid11][frame_idx] = {"human_boxes": boxes}

# ─── save one pickled .npy file ────────────────────────────────────────────
np.save(out_file, all_dets, allow_pickle=True)
print(f"✅  wrote detections for {len(all_dets)} classes → {out_file}")


class:   0%|          | 0/102 [00:00<?, ?it/s]

.ipynb_checkpoints: 0it [00:00, ?it/s]

ApplyEyeMakeup:   0%|          | 0/145 [00:00<?, ?it/s]

ApplyLipstick:   0%|          | 0/114 [00:00<?, ?it/s]

Archery:   0%|          | 0/145 [00:00<?, ?it/s]

BabyCrawling:   0%|          | 0/132 [00:00<?, ?it/s]

BalanceBeam:   0%|          | 0/108 [00:00<?, ?it/s]

BandMarching:   0%|          | 0/155 [00:00<?, ?it/s]

BaseballPitch:   0%|          | 0/150 [00:00<?, ?it/s]

In [ ]:
d = np.load(out_file, allow_pickle=True).item()
c  = next(iter(d))
v  = next(iter(d[c]))
f  = next(iter(d[c][v]))
print("sample keys:", c, v, f)
print("boxes shape:", d[c][v][f]['human_boxes'].shape)
